In [6]:
import os
import json
from pathlib import Path

import pandas as pd
from langchain.output_parsers import PydanticOutputParser
from langchain_mistralai.chat_models import ChatMistralAI
from dotenv import load_dotenv

from core import run
from prompt import DEFAULT_BASE_PROMPT, create_prompt
from schemas import Recipe

In [2]:
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

In [5]:
df = pd.read_json("data/input/recipes_v1.json")
df.head()

,title,date,tags,introduction,ingredients,direction,output
0,Creamy Mashed Potatoes,2021-03-12,"[potato, side, cheesefare]",![Creamy Mashed Potatoes](/pix/creamy-mashed-p...,The quantities here are for about four adult p...,1. Peel and cut the potatoes into medium sized...,"{'title': 'Creamy Mashed Potatoes', 'ingredien..."
1,Red Sauce (Ragu all'Italiana),2021-03-16,"[italian, sauce]",My great-grandma's red sauce. All purpose: goo...,- 1/3 lb salt pork - 2 lb chuck roast - 3 Clov...,"1. Mince the garlic, peel the carrot, peel and...","{'title': 'Red Sauce (Ragu all'Italiana)', 'in..."
2,Turmeric Flatbread,2022-08-09,"[bread, turkish]",A great companion to Turkish Red Lentil Soup. ...,- 1 cup Wheat Flour (white or whole) - 1/2 Tbs...,"1. Combine flour, turmeric, salt and baking po...","{'title': 'Turmeric Flatbread', 'ingredients':..."
3,Zurich-Style Meat Saute,2023-01-06,"[beef, cream, swiss, quick]","Originally called ""Züri Gschnätzlets"" (Zurich ...","- 600g\tBeef, finely sliced - 250g\tMushrooms,...",1. Fry the meat in butter at high heat until i...,"{'title': 'Zurich-Style Meat Saute', 'ingredie..."
4,Tuscan Style Pork Roast,2021-03-10,"[italian, pork, roast]",![tuscan-style-pork-roast](/pix/tuscan-style-p...,- 1 pork Roast - 2-3 Tbsp fresh rosemary - 8 c...,1.\tPreheat oven to 275°F (135°C) 2.\tButterfl...,"{'title': 'Tuscan Style Pork Roast', 'ingredie..."


In [7]:
llm = ChatMistralAI(api_key=MISTRAL_API_KEY, model_name="open-mixtral-8x7b")
parser = PydanticOutputParser(pydantic_object=Recipe)

In [8]:
prompt = create_prompt(DEFAULT_BASE_PROMPT, parser, df["ingredients"][0], df["direction"][0])
prompt

[HumanMessage(content='\nWhat are the ingredients and their associated quantities as well as the steps to make the recipe described by the following The quantities here are for about four adult portions. If you are planning on eating this as a side dish, it might be more like 6-8 portions. * 1kg potatoes * 200ml milk* * 200ml mayonnaise* * ~100g cheese * Garlic powder * 12-16 strips of bacon * Butter * 3-4 green onions * Black pepper * Salt  *You can play with the proportions depending on how creamy or dry you want the mashed potatoes to be. and 1. Peel and cut the potatoes into medium sized pieces. 2. Put the potatoes in a pot with some water so that it covers the potatoes and   boil them for about 20-30 minutes, or until the potatoes are soft. 3. About ten minutes before removing the potatoes from the boiling water, cut   the bacon into little pieces and fry it. 4. Warm up the milk and mayonnaise. 5. Shred the cheese. 6. When the potatoes are done, remove all water from the pot, add 

In [9]:
example = await run(llm, prompt, parser)
example

Recipe(name='Bacon Cheese Mashed Potatoes', serving_size=4, ingredients=[Ingredient(id=1, name='potatoes', quantity=1000.0, unit='grams'), Ingredient(id=2, name='milk', quantity=200.0, unit='ml'), Ingredient(id=3, name='mayonnaise', quantity=200.0, unit='ml'), Ingredient(id=4, name='cheese', quantity=100.0, unit='grams'), Ingredient(id=5, name='garlic powder', quantity=None, unit=None), Ingredient(id=6, name='bacon', quantity=None, unit=None), Ingredient(id=7, name='butter', quantity=None, unit=None), Ingredient(id=8, name='green onions', quantity=None, unit=None), Ingredient(id=9, name='black pepper', quantity=None, unit=None), Ingredient(id=10, name='salt', quantity=None, unit=None)], steps=[Step(number=1, description='Peel and cut the potatoes into medium sized pieces.', preparation_time=None, cooking_time=None, used_ingredients=[1]), Step(number=2, description='Put the potatoes in a pot with some water so that it covers the potatoes and boil them for about 20-30 minutes, or until t